ANLY 590 \
Final Project \
December 16, 2020 \
Random Forrest: Chris Fiaschetti, Ryan Callahan, Ruchikaa Kanar & Masha Gubenko

Github repo: https://github.com/mgubenko/590Project

In [82]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import keras
from keras import layers


#from google.colab import drive
import numpy as np
import pandas as pd
import scipy.io
import os
import PIL
from PIL import Image
from numpy import asarray
#import cache_magic
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import ResNet50V2
from os import makedirs
from tensorflow.keras.models import load_model
from numpy import dstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#drive.mount('/content/gdrive/')

In [3]:
# Define list that will hold all filenames in sequence
file_string_list = []
# Define list that will hold arrays of pictures' pixels in sequence
data_list = []

# Save string that is beginning of every image's filepath
dirpath='Images/'
# Save list of all folders of dog breeds
folders = os.listdir(dirpath)
# For each folder (i.e. breed)...
for folder in folders:
  # Print the folder (i.e. breed) name for tracking purposes
  print(folder)
  # Save current folder's path as separate string
  folderpath = dirpath + folder + '/'
  # Save all filenames within current folder to list
  file_list = os.listdir(folderpath)
  # For each file in this folder...
  for filename in file_list:
    # Save string that is that file's filepath
    filepath = folderpath + filename
    # Open the image at that filepath
    image = Image.open(filepath)
    # Resize image to 200x200 pixels to enable input into neural network
    image_resized = image.resize((200,200))
    # Convert this resized image to array
    data = asarray(image_resized)
    # Append this 200x200x3 array to list of all images' arrays
    data_list.append(data)

    # Save this image's folder and filename as string, which matches the folder-filename combination in the [file list-breed label] array below for later merger (i.e. we will be merging
    # image arrays and breed labels using the filenames)
    file_string = folder + '/' + filename
    # Add this image's filename to list of all filenames, which will be in same sequence as list of all image data
    file_string_list.append(file_string)
    


n02097658-silky_terrier
n02092002-Scottish_deerhound
n02099849-Chesapeake_Bay_retriever
n02091244-Ibizan_hound
n02095314-wire-haired_fox_terrier
n02091831-Saluki
n02102318-cocker_spaniel
n02104365-schipperke
n02090622-borzoi
n02113023-Pembroke
n02105505-komondor
n02093256-Staffordshire_bullterrier
n02113799-standard_poodle
n02109961-Eskimo_dog
n02089973-English_foxhound
n02099601-golden_retriever
n02095889-Sealyham_terrier
n02085782-Japanese_spaniel
n02097047-miniature_schnauzer
n02110063-malamute
n02105162-malinois
n02086079-Pekinese
n02097130-giant_schnauzer
n02113978-Mexican_hairless
n02107142-Doberman
n02097209-standard_schnauzer
n02115913-dhole
n02106662-German_shepherd
n02106382-Bouvier_des_Flandres
n02110185-Siberian_husky
n02094258-Norwich_terrier
n02093991-Irish_terrier
n02094114-Norfolk_terrier
n02109525-Saint_Bernard
n02093754-Border_terrier
n02105251-briard
n02108551-Tibetan_mastiff
n02108422-bull_mastiff
n02085936-Maltese_dog
n02093859-Kerry_blue_terrier
n02104029-kuvasz
n

In [4]:
#%cache data_l = data_list
#%cache file_string_l = file_string_list

In [5]:
# Save strings of filepaths for files that contain images' filenames and associated labels
file1 = "file_list.mat"
file2 = "test_list.mat"
file3 = "train_list.mat"
## Read in files that contain images' filenames and associated labels
mat1 = scipy.io.loadmat(file1)
mat2 = scipy.io.loadmat(file2)
mat3 = scipy.io.loadmat(file3)

In [6]:
# Save list of filenames of images in test set
test_filenames_list = mat2['file_list']
# Define list that will hold filenames in test set after they've been stripped of their triple-array packing
test_filenames = []
# For each filename in the test set...
for filename in test_filenames_list:
  # Strip off array wrapping and add to list of cleaned filenames
  test_filenames.append(filename[0][0])
# Convert filenames list to DataFrame column for concatenation
test_filenames = pd.DataFrame(np.array(test_filenames).reshape((len(test_filenames),1)))
# Convert labels list to DataFrame column for concatenation
test_labels = pd.DataFrame(mat2['labels'])
# Concatenate filenames and labels
test_file_label = pd.concat((test_labels,test_filenames),axis=1)
# Add column names to concatenated dataframe
test_file_label.columns=['label','filename']

# Save list of filenames of images in training set
train_filenames_list = mat3['file_list']
# Define list that will hold filenames in training set after they've been stripped of their triple-array packing
train_filenames = []
# For each filename in the training set...
for filename in train_filenames_list:
  # Strip off array wrapping and add to list of cleaned filenames
  train_filenames.append(filename[0][0])
# Convert filenames list to DataFrame column for concatenation
train_filenames = pd.DataFrame(np.array(train_filenames).reshape((len(train_filenames),1)))
# Convert labels list to DataFrame column for concatenation
train_labels = pd.DataFrame(mat3['labels'])
# Concatenate filenames and labels
train_file_label = pd.concat((train_labels,train_filenames),axis=1)
# Add column names to concatenated dataframe
train_file_label.columns=['label','filename']

In [7]:
train_file_label

,label,filename
0,1,n02085620-Chihuahua/n02085620_5927.jpg
1,1,n02085620-Chihuahua/n02085620_4441.jpg
2,1,n02085620-Chihuahua/n02085620_1502.jpg
3,1,n02085620-Chihuahua/n02085620_1916.jpg
4,1,n02085620-Chihuahua/n02085620_13151.jpg
...,...,...
11995,120,n02116738-African_hunting_dog/n02116738_10614.jpg
11996,120,n02116738-African_hunting_dog/n02116738_9282.jpg
11997,120,n02116738-African_hunting_dog/n02116738_6754.jpg
11998,120,n02116738-African_hunting_dog/n02116738_9333.jpg


In [8]:
### Keeping this code here in case the image stuff doesn't work out, but it's tough to use. The labels in y, y_test, and y_train are in the same order as the 
### feature data in X_train and X_test, but I don't really understand what that "feature data" is. Each image is described as a 5,073-dimensional vector 
### (X_train is 12000x5073 and X_test is 8580x5073). But if things get bleak, we can just use this code and start running models using it.

# Save filepaths that contain feature data for training and test sets
#file4 = '/content/gdrive/My Drive/train_data.mat'
#file5 = '/content/gdrive/My Drive/test_data.mat'

# Read in training and test set feature data
#mat4 = scipy.io.loadmat(file4)
#mat5 = scipy.io.loadmat(file5)

# Extract X_train and X_test from dict
#X_train = mat4['train_fg_data']
#X_test = mat5['test_fg_data']

# Extract y_train and y_test from dict
#y = mat1['labels']
#y_train = mat3['labels']
#y_test = mat2['labels']

In [9]:
# Save all image arrays as dataframe
a = pd.DataFrame(data_list)
# Rename column of image arrays
a = a.rename(columns={0:'data'})
# Add filenames, which were acquired in sequence during original for loop, as column in dataframe
a['filename'] = file_string_list
# Create dataframe of training image arrays
train_file = train_file_label.merge(a, on='filename', how='inner')
# Create dataframe of test image arrays
test_file = test_file_label.merge(a, on='filename', how='inner')

In [28]:
# Save list of training set labels
y_train_numbers = train_file.label
# Define array that will hold training set labels in softmax form
y_train = np.zeros((y_train_numbers.shape[0],120))
# For each image...
for i in range(y_train_numbers.shape[0]):
    # Flip the appropriate column's entry (each column represents a different breed) to 1
    y_train[i,y_train_numbers[i]-1] = 1
# Save list of test set labels
y_test_numbers = test_file.label
# Define array that will hold test set labels in softmax form
y_test = np.zeros((y_test_numbers.shape[0],120))
# For each image...
for i in range(y_test_numbers.shape[0]):
    # Flip the appropriate column's entry (each column represents a different breed) to 1
    y_test[i,y_test_numbers[i]-1] = 1

In [12]:
# Save series of image arrays as list
X_train = list(train_file.data)
# Remove empty 4th dimension from 7904th image
X_train[7904] = X_train[7904][:,:,0:3]
# Convert training set to numpy array and normalize variables
X_train = np.array(X_train).astype('float32') / 255.

In [13]:
# Convert test set to numpy array and normalize variables
X_test = np.array(list(test_file.data)).astype('float32') / 255.

In [ ]:
## You can use this format to show an image. This current code shows the blank 4th dimension, but you could change
# the 3 to 0, 1, or 2 to see a vague image of the collie
#image3=Image.fromarray(X_train[7904][:,:,3])
#image3.show()

In [ ]:
## At this point, the image data should be formatted properly for entry into a neural network. I've just been
# using my autoencoder from the homework to check functionality of the data format; you can delete below this.

First model is very standard - 3x3 kernels throughout, strides of 1 in both axes throughout, ReLU activation functions, padding that preserves dimensions, and a neuron sequence of 32-32-64-64-128-128-32-16.

In [39]:
# Start with empty sequential model
model = Sequential()
# Add pair of 32-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 32x32 shape
model.add(layers.Conv2D(32, (3,3), strides=(1,1), activation='relu', padding='same', input_shape=(200, 200, 3)))
model.add(layers.Conv2D(32, (3,3), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 16x16
model.add(layers.MaxPooling2D((2, 2)))
# Add pair of 64-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 16x16 shape
model.add(layers.Conv2D(64, (3,3), strides=(1,1), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3,3), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 8x8
model.add(layers.MaxPooling2D((2, 2)))
# Add pair of 128-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 8x8 shape
model.add(layers.Conv2D(128, (3,3), strides=(1,1), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 4x4
model.add(layers.MaxPooling2D((2, 2)))
# Flatten 4x4x128 network into 
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_98 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 50, 50, 128)     

In [40]:
#model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_test, y_test))

 81/188 [===========>..................] - ETA: 18:36 - loss: 4.7891 - accuracy: 0.0064

KeyboardInterrupt: 

Second model only changes the kernel size to 6x6; otherwise, we keep strides of 1 in both axes throughout, ReLU activation functions, padding that preserves dimensions, and a neuron sequence of 32-32-64-64-128-128-32-16.

In [ ]:
# Start with empty sequential model
model2 = Sequential()
# Add pair of 32-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 32x32 shape
model2.add(layers.Conv2D(32, (6,6), strides=(1,1), activation='relu', padding='same', input_shape=(200, 200, 3)))
model2.add(layers.Conv2D(32, (6,6), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 16x16
model2.add(layers.MaxPooling2D((2, 2)))
# Add pair of 64-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 16x16 shape
model2.add(layers.Conv2D(64, (6,6), strides=(1,1), activation='relu', padding='same'))
model2.add(layers.Conv2D(64, (6,6), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 8x8
model2.add(layers.MaxPooling2D((2, 2)))
# Add pair of 128-neuron hidden layers with ReLU activation function, 3x3 kernel, stride length of 1 in both dimensions, and padding that maintains 8x8 shape
model2.add(layers.Conv2D(128, (6,6), strides=(1,1), activation='relu', padding='same'))
model2.add(layers.Conv2D(128, (6,6), strides=(1,1), activation='relu', padding='same'))
# Max pooling of 2x2 sections to get dimensionality of each image down to 4x4
model2.add(layers.MaxPooling2D((2, 2)))
# Flatten 4x4x128 network into 
model2.add(layers.Conv2D(32, (6,6), strides=(1,1), activation='relu', padding='same'))
#model2.add(layers.Conv2D(16, (6,6), strides=(1,1), activation='relu', padding='same'))
model2.add(layers.Flatten())
model2.add(layers.Dense(120, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model2.summary()

In [ ]:
#model2.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_test, y_test))

We also try some transfer learning.

In [62]:
# Import library that contains VGG weights

# Construct variable that uses VGG weights for 32x32x3 input image
conv_base = VGG16(weights="imagenet",include_top=False,input_shape=(200,200,3))
# Define transfer learning model
model3 = Sequential()
# Freeze VGG weights
conv_base.trainable=False
# Add VGG weights to model
model3.add(conv_base)
# Add final dense 10-element layer that will contain softmax probabilities
model3.add(layers.GlobalMaxPool2D())
model3.add(layers.Dense(120,activation='softmax'))
# Compile model
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# Print model summary
model3.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
global_max_pooling2d_8 (Glob (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 120)               61560     
Total params: 14,776,248
Trainable params: 61,560
Non-trainable params: 14,714,688
_________________________________________________________________


In [63]:
model3.fit(X_train, y_train, epochs=1, batch_size=16, validation_data=(X_test, y_test))

750/750 [==============================] - 6180s 8s/step - loss: 4.4205 - categorical_accuracy: 0.0653 - val_loss: 3.8695 - val_categorical_accuracy: 0.1432


In [56]:
# Import library that contains VGG weights

# Construct variable that uses VGG weights for 32x32x3 input image
conv_base = Xception(weights="imagenet",include_top=False,input_shape=(200,200,3))
# Define transfer learning model
model4 = Sequential()
# Freeze VGG weights
conv_base.trainable=False
# Add VGG weights to model
model4.add(conv_base)
# Add final dense 10-element layer that will contain softmax probabilities
model4.add(layers.GlobalMaxPool2D())
model4.add(layers.Dense(120,activation='softmax'))
# Compile model
model4.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# Print model summary
model4.summary()

83689472/83683744 [==============================] - 24s 0us/step
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_max_pooling2d_6 (Glob (None, 2048)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 120)               245880    
Total params: 21,107,360
Trainable params: 245,880
Non-trainable params: 20,861,480
_________________________________________________________________


In [57]:
model4.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

375/375 [==============================] - 2149s 6s/step - loss: 1.5205 - categorical_accuracy: 0.6622 - val_loss: 1.2504 - val_categorical_accuracy: 0.7270


In [60]:
# Import library that contains VGG weights

# Construct variable that uses VGG weights for 32x32x3 input image
conv_base = ResNet50V2(weights="imagenet",include_top=False,input_shape=(200,200,3))
# Define transfer learning model
model5 = Sequential()
# Freeze VGG weights
conv_base.trainable=False
# Add VGG weights to model
model5.add(conv_base)
# Add final dense 10-element layer that will contain softmax probabilities
model5.add(layers.GlobalMaxPool2D())
model5.add(layers.Dense(120,activation='softmax'))
# Compile model
model5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# Print model summary
model5.summary()

94674944/94668760 [==============================] - 158s 2us/step
Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_max_pooling2d_7 (Glob (None, 2048)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 120)               245880    
Total params: 23,810,680
Trainable params: 245,880
Non-trainable params: 23,564,800
_________________________________________________________________


In [61]:
model5.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

375/375 [==============================] - 1590s 4s/step - loss: 9.2001 - categorical_accuracy: 0.4018 - val_loss: 5.9429 - val_categorical_accuracy: 0.5467


In [74]:
#model_little = Sequential()
#model_little.add(layers.Conv2D(32, (5,5), strides=(1,1), activation='relu', padding='same', input_shape=(200, 200, 3)))
#model_little.add(layers.MaxPooling2D((4, 4)))
#model_little.add(layers.Conv2D(16, (5,5), strides=(1,1), activation='relu', padding='same'))
#model_little.add(layers.MaxPooling2D((4, 4)))
#model_little.add(layers.Conv2D(8, (3,3), strides=(1,1), activation='relu', padding='same'))
#model_little.add(layers.MaxPooling2D((2, 2)))
#model_little.add(layers.Conv2D(4, (3,3), strides=(1,1), activation='relu', padding='same'))
#model_little.add(layers.MaxPooling2D((2, 2)))
#model_little.add(layers.Flatten())
#model_little.add(layers.Dense(120, activation='softmax'))
#model_little.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model_little.summary()
#model_little.fit(X_train, y_train, epochs=1, batch_size=32)

 15/375 [>.............................] - ETA: 5:02 - loss: 4.7943 - accuracy: 0.0021

KeyboardInterrupt: 

This is an attempt at ensemble learning with simple models. No dice.

In [75]:
def fit_model(trainX, trainy):
    # define model
    model_little = Sequential()
    model_little.add(layers.Conv2D(32, (5,5), strides=(1,1), activation='relu', padding='same', input_shape=(200, 200, 3)))
    model_little.add(layers.MaxPooling2D((4, 4)))
    model_little.add(layers.Conv2D(16, (5,5), strides=(1,1), activation='relu', padding='same'))
    model_little.add(layers.MaxPooling2D((2, 2)))
    model_little.add(layers.Conv2D(8, (3,3), strides=(1,1), activation='relu', padding='same'))
    model_little.add(layers.MaxPooling2D((2, 2)))
    model_little.add(layers.Conv2D(4, (3,3), strides=(1,1), activation='relu', padding='same'))
    model_little.add(layers.MaxPooling2D((2, 2)))
    model_little.add(layers.Flatten())
    model_little.add(layers.Dense(120, activation='softmax'))
    model_little.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_little.fit(X_train, y_train, epochs=1)
    return model_little


#makedirs('models')
# fit and save models
n_members = 5
for i in range(n_members):
    # fit model
    model = fit_model(X_train, y_train)
    # save model
    filename = 'models/model_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)

375/375 [==============================] - 384s 1s/step - loss: 4.7889 - accuracy: 0.0052
>Saved models/model_1.h5
375/375 [==============================] - 359s 958ms/step - loss: 4.7888 - accuracy: 0.0061
>Saved models/model_2.h5
375/375 [==============================] - 386s 1s/step - loss: 4.7804 - accuracy: 0.0098
>Saved models/model_3.h5
375/375 [==============================] - 361s 963ms/step - loss: 4.7886 - accuracy: 0.0060
>Saved models/model_4.h5
375/375 [==============================] - 380s 1s/step - loss: 4.7833 - accuracy: 0.0077
>Saved models/model_5.h5


But now we get serious and do an ensemble model where each of the submodels is a transfer learning model!

In [76]:
# Save Xception model
model4.save('models/model_4.h5')
# Save ResNet model
model5.save('models/model_5.h5')

In [ ]:
# This loop is here in case we want to make a loop of identical simple models in the future. Chris, maybe your computer
# is powerful enough to make this work.

#def load_all_models(n_models):
#    all_models = list()
#    for i in range(n_models):
#        # define filename for this ensemble
#        filename = 'models/model_' + str(i + 1) + '.h5'
#        # load model from file
#        model = load_model(filename)
#        # add to list of members
#        all_models.append(model)
#        print('>loaded %s' % filename)
#    return all_models
#
#n_members = 5
#members = load_all_models(n_members)
#print('Loaded %d models' % len(members))

In [79]:
# Define empty list that will hold sub-models
all_tran_models = list()
# Load Xception model
model4 = load_model('models/model_4.h5')
# Add Xception model to list
all_tran_models.append(model4)
# Load ResNet model
model5 = load_model('models/model_5.h5')
# Add ResNet model to list
all_tran_models.append(model5)

# Evaluate and print accuracy of Xception and ResNet models in isolation (for comparison against ensemble accuracy)
model4_acc = model4.evaluate(X_test,y_test)
model5_acc = model5.evaluate(X_test,y_test)
print('Xception accuracy:',model4_acc)
print('ResNet accuracy:',model5_acc)

269/269 [==============================] - 657s 2s/step - loss: 5.9429 - categorical_accuracy: 0.5467
Xception accuracy: [1.2504279613494873, 0.727039635181427]
ResNet accuracy: [5.942904472351074, 0.5467365980148315]


In [81]:
# Save VGG model to file
model3.save('models/model_3.h5')
# Load VGG model from disk
model3 = load_model('models/model_3.h5')
# Add VGG model to list of models
all_tran_models.append(model3)
# Evaluate and print performance of VGG model in isolation
model3_acc = model3.evaluate(X_test,y_test)
print('VGG accuracy:',model3_acc)

269/269 [==============================] - 1738s 6s/step - loss: 3.8695 - categorical_accuracy: 0.1432
VGG accuracy: [3.8695106506347656, 0.1432400941848755]


In [ ]:
## Again, code if we want to large number of simple, identical models later

#for model in members:
#    y_test_enc = to_categorical(y_test)
#    _, acc = model.evaluate(X_test, y_test, verbose=0)
#    print('Model Accuracy: %.3f' % acc)

In [ ]:
## Function creates multiple copies of X for evaluation
def stacked_dataset(members, inputX):
    # Define structure that will hold all predictions
    stackX = None
    # For each model contributing to ensemble...
    for model in members:
        # Make predictions using this model
        yhat = model.predict(inputX,verbose=1)
        # If stack is empty, start stack with predictions from this model
        if stackX is None:
            stackX = yhat
        # If stack is not empty, add predictions from this model to stack
        else:
            stackX = dstack((stackX, yhat))
    # Flatten predictions of all models into [observations, models x probabilities] shape
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    # Return flattened predictions
    return stackX

# Define function that fits ensemble model
def fit_stacked_model(members, inputX, inputy):
    # Create dataset of multiple copies of datset for use in ensemble model
    stackedX = stacked_dataset(members, inputX)
    # Fit ensemble model
    model = LogisticRegression()
    model.fit(stackedX, inputy)
    return model

# Use 2 aforementioned functions to create dataset shape to fit ensemble model, and then fit ensemble model
model_ensemble = fit_stacked_model(all_tran_models, X_test, np.array(y_test_numbers))

## Define function that makes predictions based on fit ensemble model
def stacked_prediction(members, model, inputX):
    # Create stacked copies of dataset
    stackedX = stacked_dataset(members, inputX)
    # Make a prediction on stacked, copied dataset using ensemble model
    yhat = model.predict(stackedX,verbose=1)
    return yhat


# Make predictions using fit ensemble model
yhat = stacked_prediction(all_tran_models, model_ensemble, X_test)
# Print accuracy of ensemble model
acc = accuracy_score(np.array(y_test_numbers), yhat)
print('Stacked Test Accuracy: %.3f' % acc)

230/269 [========================>.....] - ETA: 2:30